Basic workflow with some models
-----------------------------
Assumes recs/build_models.py has run

In [10]:
import cPickle
import sqlite3

from nltk.corpus import stopwords
from gensim import corpora, models, similarities

beerwords = corpora.Dictionary.load('beerwords.dict')
with open('corpus.pickle') as rh:
    corpus = cPickle.load(rh)
lda = models.LdaModel.load('lda.pickle')

In [40]:
from nltk.corpus import stopwords
custom_stopwords = stopwords.words('English') + [u'beer', u'one', u'like', u'well', u'really', u'very', u'it\x92s', u'it', u'beers',u'would',u'taste',u'tastes',u'get',u'i\x92m',u'quite',u'bit',u'much',u'good',u'better',u'think',u'first',u'new',u'try']

In [11]:
conn = sqlite3.connect('../src/rbeer.db')
cur = conn.cursor()

Set up our ratebeer id to corpus id and vice versa maps, for convienance 

In [12]:
cur.execute('select id, name, brewery, total_ratings from beer')
raw_map = list(enumerate(cur.fetchall()))
corpus2ratebeer = dict(raw_map)
ratebeer2corpus = {}
for corpus_id, (rb_id, name, brewery, total_ratings) in raw_map:
    ratebeer2corpus[rb_id] = (corpus_id, name, brewery, total_ratings)

grab corpus ids from rb id and confirm basic metadata

In [13]:
print ratebeer2corpus[346645]
print ratebeer2corpus[403]
print ratebeer2corpus[126190]

(278947, u'Lord Hobo Boom Sauce', 23213, 36)
(267, u'Miller Lite', 75, 1955)
(85472, u'The Kernel Export Stout London 1890', 11493, 94)


poking at Miller Lite
--------------------
grab ml's top topics and print them without stopwords
_note that various domain specific terms should probably get added to hte 

In [14]:
ml_topics = lda[corpus[267]]


In [41]:
for topic, prob in sorted(ml_topics, key = lambda x:x[1], reverse=True)[:10]:
    print "probability:", prob
    print "---"
    print filter(lambda x:x[0] not in custom_stopwords, map(lambda x:(beerwords[x[0]], x[1]),lda.get_topic_terms(topic, topn=100)))[:5]
    print "============"

probability: 0.440255523677
---
[(u'aroma', 0.0036796140866315443), (u'brew', 0.0034339150543508459), (u'light', 0.002881934064116866), (u'see', 0.0025706243002519614), (u'time', 0.0024796246442201613)]
probability: 0.207320486751
---
[(u'light', 0.033252194302523727), (u'watery', 0.023075220016330188), (u'clear', 0.02149353958946678), (u'aroma', 0.01733737244267429), (u'thin', 0.016744364050831441)]
probability: 0.0974762830548
---
[(u'sweet', 0.0089312753667533506), (u'weird', 0.0075331675655722205), (u'smells', 0.007350843071399501), (u'interesting', 0.0062618700350888868), (u'aroma', 0.0060045803194838264)]
probability: 0.0478657025998
---
[(u'little', 0.0076535695319588622), (u'bitterness', 0.0067082384178326245), (u'character', 0.0053778656717797003), (u'flavour', 0.0050577612993526902), (u'though', 0.0048262271158057016)]
probability: 0.0355497324564
---
[(u'easy', 0.06262905501754816), (u'refreshing', 0.042779998212466169), (u'drink', 0.033802444997406159), (u'summer', 0.030029

In [43]:
print "boom sauce"
for topic, prob in sorted(lda[corpus[278947]], key = lambda x:x[1], reverse=True)[:10]:
    print "probability:", prob
    print "---"
    print filter(lambda x:x[0] not in custom_stopwords, map(lambda x:(beerwords[x[0]], x[1]),lda.get_topic_terms(topic, topn=100)))[:5]
    print "============"

boom sauce
probability: 0.134785153237
---
[(u'medium', 0.017261363255937304), (u'head', 0.016130358602022825), (u'sweet', 0.014575517379677126), (u'finish', 0.013833736935477271), (u'moderate', 0.012900082538027885)]
boom sauce
probability: 0.122412015468
---
[(u'hops', 0.039841390013580016), (u'hop', 0.028569080307421515), (u'ipa', 0.026958739789170121), (u'citrus', 0.021109327489142231), (u'bitter', 0.018551288738626572)]
boom sauce
probability: 0.106526640841
---
[(u'aroma', 0.0036796140866315443), (u'brew', 0.0034339150543508459), (u'light', 0.002881934064116866), (u'see', 0.0025706243002519614), (u'time', 0.0024796246442201613)]
boom sauce
probability: 0.0946128094153
---
[(u'little', 0.0076535695319588622), (u'bitterness', 0.0067082384178326245), (u'character', 0.0053778656717797003), (u'flavour', 0.0050577612993526902), (u'though', 0.0048262271158057016)]
boom sauce
probability: 0.0758147304897
---
[(u'great', 0.011683925104252221), (u'nice', 0.0083520966000093354), (u'head', 0

In [44]:
index = similarities.MatrixSimilarity(lda[corpus])

In [46]:
cur.execute("select rbid from brewery where region LIKE '%massachusetts%'")
ma_brew = zip(*cur.fetchall())[0]
kernel_estout = lda[corpus[85472]]
like_kernel = sorted(list(enumerate(index[kernel_estout])), key = lambda x:x[1], reverse=True)

In [47]:
hits = 0
for corpus_id, score in like_kernel:
    if corpus2ratebeer[corpus_id][3] > 4 and corpus2ratebeer[corpus_id][2] in ma_brew:
        print corpus2ratebeer[corpus_id][1], corpus2ratebeer[corpus_id][0], score
        hits += 1
        if hits > 5:
            break

Paper City Fogbuster Coffee House Ale 120969 0.807707
Pioneer Extra Foreign Stout  128308 0.681164
Honest Town Capall Dorche Stout 117970 0.674146
Peoples Pint Imperial Stout 18577 0.659813
Mayflower Imperial Stout 137861 0.651553
Element Dark Element 152544 0.617399


In [48]:
boom_sauce = lda[corpus[278947]]
cur.execute("select rbid from brewery where region LIKE '%london%'")
lon_brew = zip(*cur.fetchall())[0]
like_boom = sorted(list(enumerate(index[boom_sauce])), key = lambda x:x[1], reverse=True)

In [50]:
hits = 0
for corpus_id, score in like_boom:
    if corpus2ratebeer[corpus_id][3] > 4 and corpus2ratebeer[corpus_id][2] in ma_brew:
        print corpus2ratebeer[corpus_id][1], corpus2ratebeer[corpus_id][0], score
        hits += 1
        if hits > 5:
            break

Blue Hills Quarter Mile Double IPA 240883 0.886521
Paper City Blonde Hop Monster 25768 0.871222
Mayflower 5th Anniversary Ale 198734 0.868589
Z Street IPA 13913 0.860862
Harpoon Leviathan Imperial Red Ale 114001 0.85514
Harpoon Leviathan Imperial IPA 92922 0.853753
